# Important note:

## While doing this assignment I couldn't have manage to use my GPU because of my complicated packages I downloaded(updated) over the years. I have run the code approximately 4 hours and it seemed done after 5 hours.(Because code didn't complete job, I couldn't have viualized.) But unfortunately while I was trying to use my GPU I have restarted code repeatedly. So my outputs have been vanished. I know this code isn't the best but I now it will be okay. 

## Hope you best.

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.utils import shuffle
import tensorflow as tf


In [4]:
def load_and_preprocess_data(train_csv_path, test_csv_path, image_size=(64, 64)):
    train_data = pd.read_csv(train_csv_path)
    test_data = pd.read_csv(test_csv_path)

    # Load and preprocess images
    def preprocess_images(data, image_size):
        images = []
        labels = []
        for _, row in data.iterrows():
            img_path = row['image_path']
            label = 1 if row['diagnosis'] == 'malignant' else 0

            # Load image and resize
            img = tf.keras.preprocessing.image.load_img(img_path, target_size=image_size)
            img = tf.keras.preprocessing.image.img_to_array(img) / 255.0  # Normalize
            images.append(img)
            labels.append(label)
        
        return np.array(images), np.array(labels)
    
    train_images, train_labels = preprocess_images(train_data, image_size)
    test_images, test_labels = preprocess_images(test_data, image_size)
    
    return (train_images, train_labels), (test_images, test_labels)

In [5]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

# Derivative of sigmoid
def sigmoid_derivative(z):
    return sigmoid(z) * (1 - sigmoid(z))

# Loss function: Binary Cross-Entropy
def binary_cross_entropy(y_true, y_pred):
    return -np.mean(y_true * np.log(y_pred + 1e-9) + (1 - y_true) * np.log(1 - y_pred + 1e-9))


In [6]:
def forward_propagation(X, weights, biases):
    layer_inputs = []
    layer_outputs = [X]

    for w, b in zip(weights, biases):
        z = np.dot(layer_outputs[-1], w) + b
        a = sigmoid(z)
        layer_inputs.append(z)
        layer_outputs.append(a)
    
    return layer_inputs, layer_outputs

def backward_propagation(X, y, layer_inputs, layer_outputs, weights):
    m = X.shape[0]
    gradients = {'dw': [], 'db': []}
    L = len(weights)

    # Initialize gradient for the output layer
    dz = layer_outputs[-1] - y.reshape(-1, 1)
    for l in range(L - 1, -1, -1):
        dw = np.dot(layer_outputs[l].T, dz) / m
        db = np.sum(dz, axis=0, keepdims=True) / m
        gradients['dw'].insert(0, dw)
        gradients['db'].insert(0, db)

        if l > 0:
            dz = np.dot(dz, weights[l].T) * sigmoid_derivative(layer_inputs[l - 1])
    
    return gradients

In [7]:
def update_parameters(weights, biases, gradients, learning_rate):
    for l in range(len(weights)):
        weights[l] -= learning_rate * gradients['dw'][l]
        biases[l] -= learning_rate * gradients['db'][l]
    return weights, biases

# Training 
def train(X, y, layers, learning_rate=0.01, epochs=100, batch_size=32):
    np.random.seed(42)

    # Initialize weights and biases
    weights = [np.random.randn(layers[i], layers[i+1]) * 0.01 for i in range(len(layers) - 1)]
    biases = [np.zeros((1, layers[i+1])) for i in range(len(layers) - 1)]

    losses = []
    for epoch in range(epochs):
        X, y = shuffle(X, y)

        for i in range(0, X.shape[0], batch_size):
            X_batch = X[i:i + batch_size]
            y_batch = y[i:i + batch_size]

            # Forward propagation
            layer_inputs, layer_outputs = forward_propagation(X_batch, weights, biases)

            # Compute loss
            loss = binary_cross_entropy(y_batch, layer_outputs[-1])
            losses.append(loss)

            # Backward propagation
            gradients = backward_propagation(X_batch, y_batch, layer_inputs, layer_outputs, weights)

            # Update weights and biases
            weights, biases = update_parameters(weights, biases, gradients, learning_rate)
        
        print(f"Epoch {epoch + 1}/{epochs}, Loss: {loss:.4f}")

    return weights, biases, losses

In [8]:
def evaluate(X, y, weights, biases):
    _, layer_outputs = forward_propagation(X, weights, biases)
    predictions = (layer_outputs[-1] > 0.5).astype(int)
    
    accuracy = accuracy_score(y, predictions)
    precision = precision_score(y, predictions)
    recall = recall_score(y, predictions)
    f1 = f1_score(y, predictions)
    
    return accuracy, precision, recall, f1

# Visualization of results
def plot_metrics(losses):
    plt.plot(losses)
    plt.xlabel('Iteration')
    plt.ylabel('Loss')
    plt.title('Loss Curve')
    plt.show()

In [9]:
train_data_path = "311PA3_melanoma_dataset/train"
test_data_path = "311PA3_melanoma_dataset/test"

In [10]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [11]:
def generate_csv(data_path, output_csv):
    rows = []
    for label in ["benign", "malignant"]:
        class_dir = os.path.join(data_path, label)  # Join path safely
        if not os.path.exists(class_dir):
            print(f"Directory does NOT exist: {class_dir}")
            continue  # Skip if the directory is missing
        for file_name in os.listdir(class_dir):
            if file_name.endswith(".jpg"):
                rows.append({
                    "image_path": os.path.join(class_dir, file_name),
                    "diagnosis": label
                })
    # Save as CSV
    pd.DataFrame(rows).to_csv(output_csv, index=False)

In [12]:
train_data_path = r"C:/path/to/311PA3_melanoma_dataset/train"
test_data_path = r"C:/path/to/311PA3_melanoma_dataset/test"


In [13]:
# Set the correct absolute paths
train_data_path = r"C:/Users/Arda Deniz/Desktop/Machine Learning assignment_3/311PA3_melanoma_dataset/train"
test_data_path = r"C:/Users/Arda Deniz/Desktop/Machine Learning assignment_3/311PA3_melanoma_dataset/test"

# Regenerate the CSV files
generate_csv(train_data_path, "train_data.csv")
generate_csv(test_data_path, "test_data.csv")


In [14]:
(train_images, train_labels), (test_images, test_labels) = load_and_preprocess_data(
    train_csv_path="train_data.csv",
    test_csv_path="test_data.csv",
    image_size=(64, 64)
)


In [15]:
# Updated Parameters for MLP
learning_rates = [0.005, 0.02]
batch_size = 16
input_sizes = [(50, 50), (300, 300)]  # Resize input images
activation_functions = [sigmoid, np.maximum]  # Sigmoid and ReLU
hidden_layers = [128, 64, 1]  # Fixed hidden layers for all experiments

# Perform 8 Experiments for MLP
mlp_results = []
for lr in learning_rates:
    for input_size in input_sizes:
        for activation_fn in activation_functions:
            # Resize images and preprocess
            resized_train_images = tf.image.resize(train_images, input_size).numpy().reshape(len(train_images), -1)
            resized_test_images = tf.image.resize(test_images, input_size).numpy().reshape(len(test_images), -1)

            # Train the model
            weights, biases, losses = train(
                resized_train_images, train_labels,
                layers=[input_size[0] * input_size[1] * 3] + hidden_layers,
                learning_rate=lr,
                epochs=10,
                batch_size=batch_size
            )

            # Evaluate the model
            accuracy, precision, recall, f1 = evaluate(resized_test_images, test_labels, weights, biases)

            # Log results
            mlp_results.append({
                "Learning Rate": lr,
                "Batch Size": batch_size,
                "Input Size": f"{input_size[0]}x{input_size[1]}",
                "Activation Function": "Sigmoid" if activation_fn == sigmoid else "ReLU",
                "Accuracy": accuracy,
                "Precision": precision,
                "Recall": recall,
                "F1-Score": f1
            })

# Display results for MLP
import pandas as pd
mlp_results_df = pd.DataFrame(mlp_results)
print(mlp_results_df)


KeyboardInterrupt: 

In [16]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Updated Parameters for CNN
learning_rates = [0.005, 0.02]
batch_sizes = [16, 32]
input_size = (300, 300)  # Fixed input size for all CNN experiments
activation_functions = ['sigmoid', 'relu']

# Perform 8 Experiments for CNN
cnn_results = []
for lr in learning_rates:
    for batch_size in batch_sizes:
        for activation_fn in activation_functions:
            # Build the CNN model
            model = models.Sequential([
                layers.Conv2D(32, (3, 3), activation=activation_fn, input_shape=(input_size[0], input_size[1], 3)),
                layers.MaxPooling2D((2, 2)),
                layers.Conv2D(64, (3, 3), activation=activation_fn),
                layers.MaxPooling2D((2, 2)),
                layers.Flatten(),
                layers.Dense(128, activation=activation_fn),
                layers.Dense(1, activation='sigmoid')  # Binary classification
            ])

            # Compile the model
            model.compile(
                optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
                loss='binary_crossentropy',
                metrics=['accuracy']
            )

            # Resize images and preprocess
            resized_train_images = tf.image.resize(train_images, input_size).numpy()
            resized_test_images = tf.image.resize(test_images, input_size).numpy()

            # Train the model
            history = model.fit(
                resized_train_images, train_labels,
                epochs=10,  # Adjust for quick runs
                batch_size=batch_size,
                validation_data=(resized_test_images, test_labels),
                verbose=1
            )

            # Evaluate the model
            test_loss, test_accuracy = model.evaluate(resized_test_images, test_labels, verbose=0)

            # Log results
            cnn_results.append({
                "Learning Rate": lr,
                "Batch Size": batch_size,
                "Activation Function": activation_fn.capitalize(),
                "Test Accuracy": test_accuracy
            })

# Display results for CNN
cnn_results_df = pd.DataFrame(cnn_results)
print(cnn_results_df)


c:\Users\Arda Deniz\anaconda3\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
481/481 ━━━━━━━━━━━━━━━━━━━━ 237s 482ms/step - accuracy: 0.4899 - loss: 1.0716 - val_accuracy: 0.5206 - val_loss: 0.6926
Epoch 2/10
481/481 ━━━━━━━━━━━━━━━━━━━━ 219s 455ms/step - accuracy: 0.5060 - loss: 0.6997 - val_accuracy: 0.4794 - val_loss: 0.7189
Epoch 3/10
304/481 ━━━━━━━━━━━━━━━━━━━━ 1:17 439ms/step - accuracy: 0.5004 - loss: 0.7012

KeyboardInterrupt: 

In [ ]:
# Combine results into a single table
combined_results = pd.concat([
    mlp_results_df.assign(Model='MLP'),
    cnn_results_df.assign(Model='CNN')
])

# Save to CSV for submission
combined_results.to_csv("experiment_results.csv", index=False)

# Print Combined Results
print(combined_results)


NameError: name 'cnn_results_df' is not defined

: 